In [2]:
def csv_to_r_exams(csv_file_name, scope=False, hints=False):
    """
    # The python function `csv_to_r_exams`

    This function makes it easy to create many R/Exams questions from a single CSV file.
    People with no knowledge of R/Exams or specific Exam of Testing tools, can still create many exam questions.

    # The `scope` can be:

    1. a single integer
        This will only print one exam question, based on the Pandas index number.
    2. a list of integers, p.e. [1,4,12,14]
        This will create multple exam questions, based on the Pandas index numbers.
    3. a string
        This will create only question where the `exname` is queal to the string.
    4. empty
        This will create every question that is in the CSV file

    # Only if `hints` are `True` than some hints in HTML Commend will be added to the Rmd file. This can be handy if exams will be created in RStudio from Rmd-files.

    These are the steps:

    1. Create a CSV file, or use the `csv_to_r_exams.csv` template (all should be semicolon separated).
    2. Run the `csv_to_r_exams` Python function, as the CSV-file-name as an argument and optionally a `scope`.
    3. The `csv_to_r_examas` will create one or multiple `quesion_000x.Rmd` files.
    4. You can edit the Rmd-files directly if adaption in needed (be sure not to overwrite the Rmd file again by using the `csv_to_r_exams` function again).
    5. Use RStudio to create a QTI-package that can be used in several testing applications like: Moodle, Canvas, TestVision, PDF.
    5. Future possible implementation could: Create a Python program with Python variables, that can spit out *.Rmd files with images by itself. 

    # The Meta-information:
            > # `extype:` default = `mchoice`, other options: `num` `string` `cloze` `schoice`
            > # `exname:` default = `default`, otherwise you create your own name
            > # `extol:` # only good when using `num` questions
            > # `exschuffle:` default = `TRUE`, other option: `FALSE` 
    """
    
    def create_df_with_given_scope_from_csv(csv_file_name, scope=False):
        import pandas as pd # reminder `pip install pandas`
        #from IPython.display import display # Just for Jypyter
        whole_df = pd.read_csv(csv_file_name,encoding='latin1',sep=';') 

        # descide the scope and select the df based on that
        if type(scope) == int: # A single integeger as scope, thus a DF with a sigle quesion
            #df = whole_df.loc[scope]
            df = pd.DataFrame(whole_df.loc[scope])
        if type(scope) == str: # This is a string, so filter on the `exname`
            filt = whole_df['exname'] == scope
            df = whole_df[filt]
        if type(scope) == list:
            df = whole_df.iloc[scope, :]
        if not scope: 
            df = whole_df
        return df

    def write_single_question_to_Rmd(csv_file_name, index, hints=False):
        import pandas as pd # reminder `pip install pandas`
        #from IPython.display import display # Just for Jypyter
        df = pd.read_csv(csv_file_name,encoding='latin1',sep=';') 
        #display(df)
        text = ''

        def create_r_code_text(): 
            # Set starting variables
            

            if str(df.loc[index,'r-code']) == "nan": # Als er r-code staat
                r_code_text = '```{r data generation, echo = FALSE, results = "hide"}\n'
                r_code_text += '## parameters\n'
                r_code_text += '## solution\n'
                r_code_text += '```'
            else:
                r_code_text = df.loc[index,'r-code']
                multiple_lines = r_code_text.split("\\n")
                r_code_text= ""
                for line in multiple_lines:
                    r_code_text += line + '\n'   
                #r_code_text = single_lines_r_code_text
                #for single_line in single_lines_r_code_text:
                #    r_code_test = "blabla"
                #    #r_code_text = single_line + "\k"
            return r_code_text

        text += create_r_code_text()

        def create_question_text(index):
            question_text = "\n\nQuestion\n"
            question_text += "========\n"
            question_text += df.loc[index,'Question']
            return question_text

        text += create_question_text(index) 

        def create_answer_text_and_exsolution_string():
            answer_text = "\n\nAnswerlist\n"
            answer_text += "----------\n"
            correct_string = []
            columns = (len(df.columns) - 7) / 3 # The Amount of Columns for possible options. Default is 30 columns = 10 possible options 
            columns += 1
            columns = int(columns)
            
            for number in range (1, columns):
                if not str(df.loc[index, ('Answerlist_' + str(number))]) == str("nan"): 
                    answer_text += "* " + str(df.loc[index, ('Answerlist_' + str(number))]) + "\n"
                if not str(df.loc[index, ('exsolution_' + str(number))]) == str("nan"): # If the field is NOT empty.
                    correct_string.append(int(df.loc[index, ('exsolution_' + str(number))]))
   
  
            return answer_text, correct_string


        answer_text, correct_string = create_answer_text_and_exsolution_string()
        text += answer_text

        def create_solution_text():
            solution_text = ''
            answer_text = ''
            solution_text += "\nSolution\n"
            solution_text += "========\n"
            if not str(df.loc[index,'exsolution']) == str("nan"): 
                solution_text += str(df.loc[index,'exsolution']) + "\n"
            
            # Loop trough all Answers
            columns = (len(df.columns) -7) / 3 # The Amount of Columns for possible options. Default is 30 columns = 10 possible options 
            columns = int(columns)
            columns += 1
            columns = int(columns)
            
            for number in range (1, columns):
                if not str(df.loc[index, ('Answerlist_' + str(number))]) == str("nan"): 
                    answer_text += "* " + str(df.loc[index, ('Answerlist_' + str(number))]) + "\n"
                #if not str(df.loc[index, ('Solution_1')]) == str("nan") and str(df.loc[index, ('Solution_2')]) == str("nan"): # If Solution_1 is filled in and Solution_2 is empty, don't print an asterisks
                #    solution_text += str(df.loc[index, ('Solution_1')]) + "\n"
                #    break
                if not str(df.loc[index, ('Solution_' + str(number))]) == str("nan"): # if next Solution emtpy then quit
                    solution_text += "* " + str(df.loc[index, ('Solution_' + str(number))]) + "\n"
            return solution_text

        text += create_solution_text()
        
        # Add all the Meta-information
        def create_meta_information(correct_string):
            exsolution = ''
            meta_text = ''
            meta_text += "\nMeta-information\n"
            meta_text += "================\n"
            if correct_string:
                meta_text += "exsolution: "
                for result in correct_string:
                    meta_text += str(result)
                #meta_text += str(correct_string) + "\n"
                meta_text += "\n"
            else:
                meta_text += "exsolution: " + df.loc[index,'exsolution'] + "\n"
            if str(df.loc[index,'extype']) == "nan":
                meta_text += "extype: mchoice"
            else:
                meta_text += "extype: " + str(df.loc[index,'extype'])

            if str(df.loc[index,'exname']) == "nan":
                meta_text += "\nexname: default"
            else:
                meta_text += "\nexname: " + str(df.loc[index,'exname'])

            if str(df.loc[index,'exshuffle']) == "nan":        
                meta_text += "\nexshuffle: TRUE"
            elif str(df.loc[index,'exshuffle']).lower() == "true":
                meta_text += "\nexshuffle: TRUE"
            elif str(df.loc[index,'exshuffle']) == "1":
                meta_text += "\nexshuffle: TRUE"
            elif str(df.loc[index,'exshuffle']).lower() == "false":
                meta_text += "\nexshuffle: FALSE"
            elif str(df.loc[index,'exshuffle']) == "0":
                meta_text += "\nexshuffle: FALSE"

            if not str(df.loc[index,'extol']) == "nan":
                meta_text += "\nextol: " + str(df.loc[index,'extol'])

            return meta_text

        text += create_meta_information(correct_string)

        def create_hints_text():
            hints = "\n\n\n<!--" 
            hints += "\n> library('exams')"
            hints += "\n> e <- exams2html('yourfilename.Rmd')"
            hints += "\n> exams_metainfo(e)"
            hints += "\n> exams2pdf('yourfilename.Rmd')"
            hints += "\n> nops2pdf('yourfilename.Rmd')"
            hints += "\nmyexam <- list("
            hints += "\n     'question_0001.Rmd',"
            hints += "\n     'question_0002.Rmd',"
            hints += "\n     'question_0003.Rmd'"
            hints += "\n)"
            hints += "\n\n> exams2testvions(myexam)"
            hints += "\n> exams2html(myexam)"
            hints += "\n-->"
            return hints
        
        if hints == True:
            text += create_hints_text()
   
        # Write the file
        def write_to_file(index):
            file_index = ''
            if index < 10:
                file_index = str('000') + str(index)
            elif index < 100:
                file_index = str('00') + str(index)
            elif index < 1000:
                file_index = str('0') + str(index)
            else:
                file_index = str(index)  
            file = open('question_' + str(file_index) + '.Rmd', 'w', encoding='utf-8')
            file.writelines(text)
            file.close()
        
        write_to_file(index)


    import pandas as pd
    df = pd.DataFrame(create_df_with_given_scope_from_csv(csv_file_name, scope))
    for single_index in df.index:
        write_single_question_to_Rmd(csv_file_name, single_index)
    return df

csv_file_name = "R-exams_template_xlsx_to_csv_2021-08-06.csv"
#scope = [0, 2, 4]
#scope = 1
#scope = False
#scope = 'k3' # this is an `exname`
from IPython.display import display
display(csv_to_r_exams(csv_file_name, False, True))

#help(csv_to_r_exams)

number: 1
number: 2
number: 3
number: 4
number: 5
number: 6
number: 7
number: 8
number: 9
number: 10
<p style='font-color: red'>correct_string = [0, 0, 0, 1, 1]</p>
number: 1
number: 2
number: 3
number: 4
number: 5
number: 6
number: 7
number: 8
number: 9
number: 10
<p style='font-color: red'>correct_string = [0, 0, 0]</p>
number: 1
number: 2
number: 3
number: 4
number: 5
number: 6
number: 7
number: 8
number: 9
number: 10
<p style='font-color: red'>correct_string = [1, 0, 1]</p>
number: 1
number: 2
number: 3
number: 4
number: 5
number: 6
number: 7
number: 8
number: 9
number: 10
<p style='font-color: red'>correct_string = [1, 0]</p>
number: 1
number: 2
number: 3
number: 4
number: 5
number: 6
number: 7
number: 8
number: 9
number: 10
<p style='font-color: red'>correct_string = [1, 0]</p>
number: 1
number: 2
number: 3
number: 4
number: 5
number: 6
number: 7
number: 8
number: 9
number: 10
<p style='font-color: red'>correct_string = [1, 0, 0, 0]</p>


,Question,extype,exsolution,exname,extol,exshuffle,r-code,Answerlist_1,exsolution_1,Solution_1,...,Solution_7,Answerlist_8,exsolution_8,Solution_8,Answerlist_9,exsolution_9,Solution_9,Answerlist_10,exsolution_10,Solution_10
0,Gaat de Python functie en CSV werken?,NaN,NaN,NaN,NaN,NaN,NaN,"Ja, het gaat wel lukken.",0,Jaaa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Who live in the woods near the willow tree,NaN,NaN,NaN,NaN,NaN,NaN,Sexual harrasment panda,0,Goed sauspark gekeken,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Wat is pandas,NaN,NaN,NaN,NaN,NaN,NaN,Een soort pinda,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Wat is Samson,schoice,NaN,samson,NaN,NaN,NaN,Een hond,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wat is gertje?,NaN,NaN,samson,NaN,NaN,NaN,Een man,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Wat is ${`r a`} + {`r b`}$,num,`r res`,rcode,NaN,4.0,"```{r data generation, echo = FALSE, results =...",${`r a`} + {`r b`} = {`r res + 0`}$,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
